In [1]:
import os

In [2]:
# Check present working directory
%pwd

'c:\\Sources\\End_to_end_MLOPS_project\\research'

In [3]:
# Change present working directory to thr root directory
os.chdir('../')

In [4]:
%pwd

'c:\\Sources\\End_to_end_MLOPS_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

[2025-02-02 21:07:57,272: INFO: __init__: Loaded constants. CONFIG_FILE_PATH=config\config.yaml]


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-01 18:35:29,260: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-01 18:35:29,269: INFO: common: yaml file: params.yaml loaded successfully]


[2025-02-01 18:35:29,282: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-01 18:35:29,284: INFO: common: created directory at: artifacts]
[2025-02-01 18:35:29,285: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-01 18:35:30,945: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 100356
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7eb16441bd00dc318bbc922b362147e8ed3565dc5d64a9c26ca21170ebbd49e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9B63:185322:5D36EA:68A2D1:679EBDD2
Accept-Ranges: bytes
Date: Sun, 02 Feb 2025 00:35:30 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-kigq8000063-CHI
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1738456531.605875,VS0,VE218
Vary: Authorization,Accept-